### 3. KOBIS OPENAPI

영화관 입장권 통합 전산망 OPENAPI를 사용하여 일일 박스오피스 자료를 가져오고 JSON파일로 저장

# title

In [1]:
import os
import requests
import json
import calendar
from tqdm import tqdm
from tqdm import tqdm_notebook
from datetime import date, timedelta

key='ad816991534afeaef71f07e7336b0d61'
movie_info_path='/root/movie/data/'

폴더가 존재하는지 확인한 뒤 없으면 생성하는 함수

In [2]:
def check_dir(start_date,end_date):    
    start_year = int(start_date.year)
    end_year = int(end_date.year)
    year = start_year
    for i in tqdm(range(start_year,end_year+1)):        
        if(os.path.isdir(movie_info_path+str(year))!=1):
            os.mkdir(movie_info_path+str(year)+'/')
            print("create "+str(year)+" folder") 
        year += 1

일일 박스오피스 자료를 찾고 없으면 API를 통해 자료를 받아온다

파일은 JSON 타입으로 해당 폴더에 저장

In [3]:
def daily_box_office_to_json(start_date,end_date):    
    tmp_date = start_date
    startDt = int(str(start_date)[0:4]+str(start_date)[5:7]+str(start_date)[8:10])
    endDt = int(str(end_date)[0:4]+str(end_date)[5:7]+str(end_date)[8:10])
    days = end_date - start_date
    for i in tqdm(range(0,days.days+1)):
        targetDt = int(str(tmp_date)[0:4]+str(tmp_date)[5:7]+str(tmp_date)[8:10])
        
        if(os.path.isfile("/root/movie/data/"+str(targetDt)[0:4]+'/'+str(targetDt)+'.json')!=1):
            url='http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key='+key+'&targetDt='+str(targetDt)
            res = requests.get(url)
            text = res.text
            d = json.loads(text)
            with open('data/'+str(targetDt)[0:4]+'/'+str(targetDt)+'.json', 'w', encoding="utf-8") as make_file:
                json.dump(d, make_file, ensure_ascii=False, indent="\t")
        tmp_date += timedelta(1)

샘플 데이터로 2017년 전체 자료를 받아온다

In [4]:
start_date = date(2017,1,1)
end_date = date(2017,12,31)
check_dir(start_date,end_date)
daily_box_office_to_json(start_date,end_date)

100%|██████████| 365/365 [00:00<00:00, 62928.35it/s]


In [5]:
import pandas as pd
import requests
import datetime
import json
from tqdm import tqdm_notebook
from dateutil.relativedelta import relativedelta

my_date = datetime.date(2017, 1, 1)

In [6]:
start_date = date(2017,1,1)
targetDt = str(start_date)[0:4]+str(start_date)[5:7]+str(start_date)[8:10]
with open('data/'+str(targetDt)[0:4]+'/'+str(targetDt)+'.json', 'r', encoding="utf-8") as read_file:
    d = json.load(read_file)
d

{'boxOfficeResult': {'boxofficeType': '일별 박스오피스',
  'showRange': '20170101~20170101',
  'dailyBoxOfficeList': [{'rnum': '1',
    'rank': '1',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20161725',
    'movieNm': '마스터',
    'openDt': '2016-12-21',
    'salesAmt': '4399793100',
    'salesShare': '46.3',
    'salesInten': '-183705150',
    'salesChange': '-4',
    'salesAcc': '44184434165',
    'audiCnt': '506299',
    'audiInten': '-31485',
    'audiChange': '-5.9',
    'audiAcc': '5441619',
    'scrnCnt': '1245',
    'showCnt': '5721'},
   {'rnum': '2',
    'rank': '2',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20163183',
    'movieNm': '로그 원: 스타워즈 스토리',
    'openDt': '2016-12-28',
    'salesAmt': '1612424700',
    'salesShare': '17.0',
    'salesInten': '-351990650',
    'salesChange': '-17.9',
    'salesAcc': '6739961794',
    'audiCnt': '173451',
    'audiInten': '-40093',
    'audiChange': '-18.8',
    'audiAcc': '769938',
    'scrnCnt':

In [7]:
def daily_boxoffice(start_date, end_date) :
    
    final_list = []

    for single_date in tqdm_notebook(pd.date_range(start_date, end_date)) :
        targetDt = str(single_date)[0:4]+str(single_date)[5:7]+str(single_date)[8:10]
        with open('data/'+str(targetDt)[0:4]+'/'+str(targetDt)+'.json', 'r', encoding="utf-8") as read_file:
            d = json.load(read_file)

        for item in d["boxOfficeResult"]["dailyBoxOfficeList"] :
            value_list = []
            key_list = []
            for key, value in item.items() :
                key_list.append(key)
                value_list.append(value)

            value_list.append(single_date)
            key_list.append('CurrentDate')
            final_list.append(value_list)

    return pd.DataFrame(final_list, columns=key_list)

In [8]:
start_date = date(2017,1,1)
end_date = date(2017,12,31)
a = daily_boxoffice(start_date, end_date)

### 일별 박스 오피스를 월별로 추출, 영화코드 리스트 추출 ###ok


In [9]:
movie_code_list_all=pd.DataFrame()
for i in tqdm_notebook(range(12)):
    start_date = my_date + relativedelta(months=i)
    end_date = start_date+ relativedelta(months=1) - datetime.timedelta(days=1)
    
    daily_boxoffice_data = daily_boxoffice(start_date, end_date)
    startdate=start_date.strftime('%Y%m%d')
    
    enddate=end_date.strftime('%Y%m%d')
    
    movie_code_list = pd.DataFrame({'movieCd' :daily_boxoffice_data.movieCd.drop_duplicates(), 'movieNm':daily_boxoffice_data.movieNm.drop_duplicates(), 'director':'','Rate' : 0})
    
    
    daily_boxoffice_data.to_csv("daliy_boxoffice_data_"+startdate+"~"+enddate+".csv", index=False)
    movie_code_list.to_csv("movie_code_list_"+startdate+"~"+enddate+".csv", index=False)
    
    movie_code_list_all = movie_code_list.append(movie_code_list_all)
    movie_code_list_all = movie_code_list_all.drop_duplicates()
    movie_code_list_all.to_csv("movie_code_list_all.csv", index=False)

--------------------------------------------------------------
영화 코드를 통해 영화 정보 파일 불러오기
--------------------------------------------------------------

In [10]:
def movie_info_api(df):
    for i in range(0,len(df['movieCd'])):
        movieCd = str(df['movieCd'][i])

        if(os.path.isfile("/root/movie/movie_info/"+movieCd+".json")):
            print(movieCd+".json already exist!")

        else:
            url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key='\
                +key+'&movieCd='+movieCd    
            res = requests.get(url)
            text = res.text
            d = json.loads(text)
            print(movieCd+".json is created!")
            with open('movie_info/'+movieCd+'.json', 'w', encoding="utf-8") as make_file:
                json.dump(d, make_file, ensure_ascii=False, indent="\t")

In [11]:
df = pd.read_csv('movie_code_list_all.csv')
movie_info_api(df)

20165748.json already exist!
20173581.json already exist!
20178581.json already exist!
20161762.json already exist!
20040695.json already exist!
20168221.json already exist!
20179942.json already exist!
20175121.json already exist!
20174622.json already exist!
20080349.json already exist!
20175223.json already exist!
20174073.json already exist!
20170272.json already exist!
20175441.json already exist!
20175903.json already exist!
20174585.json already exist!
20173693.json already exist!
20174846.json already exist!
20167904.json already exist!
20170402.json already exist!
20170443.json already exist!
20150976.json already exist!
20175481.json already exist!
20176367.json already exist!
20000067.json already exist!
20170590.json already exist!
20179462.json already exist!
20030404.json already exist!
20174965.json already exist!
20176082.json already exist!
20176022.json already exist!
20176049.json already exist!
20176947.json already exist!
20177479.json already exist!
20172742.json 

In [12]:
#영화데이터  : txt 로 저장 함수 정의
def movie_data(lists) :    
    dict_list = dict()

    for movie in lists :
        with open('movie_info/'+movie+'.json', 'r', encoding="utf-8") as read_file:
            d = json.load(read_file)
        
        dict_list[movie] = d['movieInfoResult']['movieInfo'] #dict_list[movie] 가 있어야 append형식으로 됨. 없으면 마지막것만 추출
    return dict_list

In [13]:
movie_info=movie_data(movie_code_list_all.movieCd) 
with open('movie_info.txt', 'w', encoding='utf-8') as outfile:   #딕셔너리를 json으로 저장
    json.dump(movie_info, outfile, ensure_ascii=False)

In [16]:
#movie_info의  director를 뽑아서 movie_code_list에 넣기
movie_info_keys=list(movie_info.keys())
len(movie_info_keys)

284

In [64]:
movie_code_list_all = pd.read_csv("movie_code_list_all.csv")
for i in range(len(movie_code_list_all)):
    with open('movie_info/'+str(movie_code_list_all['movieCd'][i])+'.json', 'r', encoding="utf-8") as make_file:
        d = json.load(make_file)
    if(d['movieInfoResult']['movieInfo']['directors'] != []):
        movie_code_list_all['director'][i]=d['movieInfoResult']['movieInfo']['directors'][0]['peopleNm']
        
    else:
        movie_code_list_all['director'][i] = '---'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [65]:
# movie_code_list_all['director'][1]
movie_code_list_all['director']

0             장창원
1             장항준
2         케네스 브래너
3             김홍선
4            길 정거
5          잭 스나이더
6        도로타 코비엘라
7        타이카 와이티티
8        크리스토퍼 랜던
9           롭 라이너
10       우테 폰 뮌쇼폴
11         벤자민 레너
12            유영의
13            조영준
14        후쿠다 유이치
15     요네바야시 히로마사
16       마이클 쿠에스타
17            김현호
18        데이미언 셔젤
19            양우석
20         라이언 존슨
21            김용화
22        저스틴 채드윅
23         고로 사쿠라
24         이와이 슌지
25            장준환
26       마이클 그레이시
27        리차드 커티스
28       유야마 쿠니히코
29          호거 태프
          ...    
254        제임스 폴리
255           장예모
256       고어 버빈스키
257           마대윤
258    알베르토 로드리게즈
259           이주영
260           김준성
261          멜 깁슨
262           조의석
263       가렛 에드워즈
264        가스 제닝스
265           박정우
266      유야마 쿠니히코
267      유야마 쿠니히코
268           홍지영
269      야쿠와 신노스케
270         롭 라이너
271          켄 로치
272           김진혁
273         모튼 틸덤
274           주지홍
275     알렉세이 트시칠린
276           김태용
277        저스틴 커젤
278      로

In [63]:
with open('movie_info/'+'20179807'+'.json', 'r', encoding="utf-8") as make_file:
    d = json.load(make_file)
d['movieInfoResult']['movieInfo']['directors']

[]

In [66]:
movie_code_list_all.to_csv("movie_code_add_director.csv")

In [ ]:
############### 네이버 api로 별점 추출, def 이용_1

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import json
import re
import requests
import pandas as pd

naver_client_id = "nA8Iwm4HJ6EM2W1dueU9"
naver_client_secret = "HaEPQCPbyQ"

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
def searchByTitle(title): #네이버 api
    myurl = 'https://openapi.naver.com/v1/search/movie.json?display=100&query=' + quote(title)
    request = urllib.request.Request(myurl)
    request.add_header("X-Naver-Client-Id",naver_client_id)
    request.add_header("X-Naver-Client-Secret",naver_client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        d = json.loads(response_body.decode('utf-8'))
        if (len(d['items']) > 0):
            return d['items']
        else:
            return None 
    else:
        print("Error Code:" + rescode)

In [ ]:
#requests data 불러오기
def findItemByInput(items):
    a = {}
    n=len(items)
    #print('n = ',n)
    for index, item in enumerate(items):
        navertitle = cleanhtml(item['title'])
        naversubtitle = cleanhtml(item['subtitle'])
 #       naverpubdate = cleanhtml(item['pubDate'])
 #       naveractor = cleanhtml(item['actor'])
        naverlink = cleanhtml(item['link']) # 있어야함
        naverdirecter=cleanhtml(item['director'])
        naveruserScore = cleanhtml(item['userRating'])
 
        navertitle1 = navertitle.replace(" ","")
        navertitle1 = navertitle1.replace("-", ",")
        navertitle1 = navertitle1.replace(":", ",")
 
        spScore = getSpecialScore(naverlink)  #기자 평론가 평점을 얻어 옵니다
        
        naverid = re.split("code=", naverlink)[1]  #네이버가 다루는 영화 고유 ID를 얻어 옵니다
   #     print('naveruserScore',naveruserScore)
#         print(naveruserScore)
#         print(type(naveruserScore))
        a[naverdirecter[:3]] = naveruserScore
      #  print(naverid, navertitle,naveruserScore, spScore)
#         print(index, navertitle, naversubtitle, naveruserScore, spScore)
    return a

In [ ]:
def getInfoFromNaver(searchTitle):
    items = searchByTitle(searchTitle)
 
    if (items != None):
        b = findItemByInput(items)
    else:
        print("No result") 
    
    print(b)
    return b

In [ ]:
def get_soup(url):
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'lxml')
    return soup
 
#기자 평론가 평점을 얻어 옵니다
def getSpecialScore(URL):
    soup = get_soup(URL)
    scorearea = soup.find_all('div', "spc_score_area")
    newsoup = BeautifulSoup(str(scorearea), 'lxml')
    score = newsoup.find_all('em')
    if (score and len(score) > 5):
        scoreis = score[1].text + score[2].text + score[3].text + score[4].text
        return float(scoreis)
    else:
        return 0.0

In [ ]:
i = movie_code_list_all['movieNm'][1]

In [ ]:
name=movie_code_list_all.iloc[3,3]
name

In [ ]:
b=[]
# for r in tqdm_notebook(range(3,4)):
# print('r = ',r)

name=movie_code_list_all['movieNm'][2]
print('name = ',name)  

#     a = getInfoFromNaver(name)   #이 값을 변수에 넣을 수가 없음 ㅠㅠ 대입이 안됨

items = searchByTitle(name)
for i in range(len(items)):
    print("%d : %s",i, items)
    if (items != None):
        b.append(findItemByInput(items))
    else:
        print("No result")

# movie_code_list_all

In [ ]:
b

In [ ]:
c = {}
for r in tqdm_notebook(range(10)):
    print('r = ',r)
    
    name=movie_code_list_all['movieNm'][r]
    print('name = ',name)  
    
    c[name] = getInfoFromNaver(name)    #이 값을 변수에 넣을 수가 없음 ㅠㅠ 대입이 안됨

In [ ]:
c

In [ ]:
movie_code_list_all['movieNm']

In [ ]:
type(movie_code_list_all)

In [ ]:
for i in range(0,len(c)):
    m_Nm = movie_code_list_all[i]['movieNm']
    m_dr = movie_code_list_all[i]['director']
    if(movie_code_list_all[i]['Rate'] == 0):
        for j in range(0, len(c[m_Nm])):
            if(m_dr[:3] == c[m_Nm][m_dr[:3]]):
                movie_code_list_all[i]['Rate'] = c[m_Nm][j]
            else:
                movie_code_list_all[i]['Rate'] = 0
                

In [ ]:
movie_code_list_all

In [ ]:
############### 네이버 api로 별점 추출, def 이용_2
import os
import sys
import urllib.request
import json

client_id = "nA8Iwm4HJ6EM2W1dueU9"
client_secret = "HaEPQCPbyQ"

with open('movie_info.txt', 'w', encoding='utf-8') as outfile:   
    json.dump(movie_info, outfile, ensure_ascii=False)

from IPython.display import clear_output

def naver_starscore():
    for i in tqdm_notebook(range(len(movie_code_list_all))) : 
        url = 'https://openapi.naver.com/v1/search/movie.json'
        dic = {
            'query' : movie_code_list_all.iloc[i, 3],
            'display' : 100
        }

        headers = {
            'X-Naver-Client-Id' : 'API Client ID',
            'X-Naver-Client-Secret' : 'API Client Secret'
        }
        
        request = requests.get(url, headers=headers, dic=payload)        
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        
        if(rescode==200):
            response_body = response.read()
            js_naver = json.loads(response_body)
            print(l, 'th = request success')
            print('js_naver = \n', js_naver)

            if js_naver['total']==1: #영화명 1개, 별점 추출                
                if movie_code_list_all.iloc[l,3].replace(' ', '') == js_naver['items'][0]['title'][3:-4].replace(' ', ''):
                    movie_code_list_all.iloc[l,0] = js_naver['items'][0]['userRating']
                    print(' 영화명1개, 일치,  okokokokkok')
                else:                
                    print(' 영화명1개, 불일치')

            else:  #영화명 여러개 
             #   print('js_naver[items].count(title) = ',js_naver['items'].count('title'))
                for j in range(1,js_naver['items'].count('title')):
                 #   print('j = ', j)
                    for k in range(80):
                 #       print('k = ',k)
              #          print('js_naver director =',js_naver['items'][j]['director'][:3],'movie_code_list_all.iloc[k,1] = ',movie_code_list_all.iloc[k,1])
                        if js_naver['items'][j]['director'][:3] == movie_code_list_all.iloc[k,1]:                        
                            movie_code_list_all.iloc[l,0] = js_naver['items'][0]['userRating']
                #            print('영화여러개, 감독 일치, okokokok')
                        else:
                            movie_code_list_all.iloc[l,0] = 0
                   #         print('영화여러개, 감독불일치, 별점 0')
        else:
            print("Error Code:" + rescode)

In [ ]:
movie_code_list_all

In [ ]:
############### 네이버 api로 별점 추출, display = 100 으로 수정중
import os
import sys
import urllib.request

client_id = "nA8Iwm4HJ6EM2W1dueU9"
client_secret = "HaEPQCPbyQ"

with open('movie_info.txt', 'w', encoding='utf-8') as outfile:   
    json.dump(movie_info, outfile, ensure_ascii=False)

for i in tqdm_notebook(range(len(movie_code_list_all))) :  
    url = 'https://openapi.naver.com/v1/search/movie.json'
    dic = {
        'query' : movie_code_list_all.iloc[i, 3],
         'display' : 100
    }
    headers = {
         'X-Naver-Client-Id' : 'API Client ID',
          'X-Naver-Client-Secret' : 'API Client Secret'
       }    
    request = requests.get(url, headers=headers, params=dic)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()        
    
    if(rescode==200):
        response_body = response.read()
        js_naver = json.loads(response_body)
        print(l, 'th = request success')
        print('js_naver = \n', js_naver)
        
        if js_naver['total']==1: #영화명 1개, 별점 추출                
            if movie_code_list_all.iloc[l,3].replace(' ', '') == js_naver['items'][0]['title'][3:-4].replace(' ', ''):
                movie_code_list_all.iloc[l,0] = js_naver['items'][0]['userRating']
                print(' 영화명1개, 일치,  okokokokkok')
            else:                
                print(' 영화명1개, 불일치')
                
        else:  #영화명 여러개 
            print('js_naver[items].count(title) = ',js_naver['items'].count('title'))
            for j in range(1,js_naver['items'].count('title')):
                print('j = ', j)
                for k in range(80):
                    print('k = ',k)
                    print('js_naver director =',js_naver['items'][j]['director'][:3],'movie_code_list_all.iloc[k,1] = ',movie_code_list_all.iloc[k,1])
                    if js_naver['items'][j]['director'][:3] == movie_code_list_all.iloc[k,1]:                        
                        movie_code_list_all.iloc[l,0] = js_naver['items'][0]['userRating']
            #            print('영화여러개, 감독 일치, okokokok')
                    else:
                        movie_code_list_all.iloc[l,0] = 0
               #         print('영화여러개, 감독불일치, 별점 0')
    else:
        print("Error Code:" + rescode)

In [ ]:
movie_code_list_all

In [ ]:
############### 네이버 api로 별점 추출, display=10
import os
import sys
import urllib.request

client_id = "nA8Iwm4HJ6EM2W1dueU9"
client_secret = "HaEPQCPbyQ"

with open('movie_info.txt', 'w', encoding='utf-8') as outfile:   
    json.dump(movie_info, outfile, ensure_ascii=False)

for l in tqdm_notebook(range(len(movie_code_list_all))):
    m=movie_code_list_all.iloc[l,3] #영화명이 iloc[l,3]
    encText = urllib.parse.quote(m)
    url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText # json 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
    if(rescode==200):
        response_body = response.read()
        js_naver = json.loads(response_body)
        print(l, 'th = request success')
        print('js_naver = \n', js_naver)
        
        if js_naver['total']==1: #영화명 1개, 별점 추출                
            if movie_code_list_all.iloc[l,3].replace(' ', '') == js_naver['items'][0]['title'][3:-4].replace(' ', ''):
                movie_code_list_all.iloc[l,0] = js_naver['items'][0]['userRating']
                print(' 영화명1개, 일치,  okokokokkok')
            else:                
                print(' 영화명1개, 불일치')
                
        else:  #영화명 여러개 
            print('js_naver[items].count(title) = ',js_naver['items'].count('title'))
            for j in range(1,js_naver['items'].count('title')):
                print('j = ', j)
                for k in range(80):
                    print('k = ',k)
                    print('js_naver director =',js_naver['items'][j]['director'][:3],'movie_code_list_all.iloc[k,1] = ',movie_code_list_all.iloc[k,1])
                    if js_naver['items'][j]['director'][:3] == movie_code_list_all.iloc[k,1]:                        
                        movie_code_list_all.iloc[l,0] = js_naver['items'][0]['userRating']
            #            print('영화여러개, 감독 일치, okokokok')
                    else:
                        movie_code_list_all.iloc[l,0] = 0
               #         print('영화여러개, 감독불일치, 별점 0')
    else:
        print("Error Code:" + rescode)

In [ ]:
movie_code_list_all

In [ ]:
movie_code_list_all.to_csv("movie_code_list_all.csv", index=False)